In [ ]:
# ЯЧЕЙКА 1: Установка зависимостей с ChromaDB
import os
import sys
import subprocess

# Клонируем проект если нужно
if not os.path.exists('/content/ai-seo-architects'):
    subprocess.run(['git', 'clone', 'https://github.com/Andrew821667/ai-seo-architects.git', '/content/ai-seo-architects'])
    print('✅ Проект клонирован')

# Переходим в директорию проекта
os.chdir('/content/ai-seo-architects')
sys.path.append('/content/ai-seo-architects')

# Устанавливаем зависимости с ChromaDB
dependencies = [
    'openai==1.6.1',
    'langchain==0.1.7', 
    'langchain-openai==0.0.8',
    'langgraph==0.0.45',
    'chromadb==0.4.15',
    'numpy==1.25.2',
    'nest-asyncio==1.5.8'
]

print('📦 Установка зависимостей с ChromaDB:')
for dep in dependencies:
    subprocess.run(['pip', 'install', '-q', dep])
    print(f'✅ {dep}')

import asyncio
import nest_asyncio
nest_asyncio.apply()

print('\n🎯 Окружение настроено с ChromaDB')

In [ ]:
# ЯЧЕЙКА 2: Настройка OpenAI API ключа
import os
from google.colab import userdata
import getpass

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
        print('✅ OpenAI API ключ загружен из Colab secrets')
    else:
        raise Exception('API ключ не найден в secrets')
except:
    OPENAI_API_KEY = getpass.getpass('Введите OpenAI API ключ: ')
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    print('✅ API ключ установлен')

# Тестируем подключение
import openai
client = openai.OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role': 'user', 'content': 'Тест ChromaDB'}],
    max_tokens=10
)
print(f'✅ OpenAI API работает: {response.choices[0].message.content}')

In [ ]:
# ЯЧЕЙКА 3: Инициализация ChromaDB Knowledge Manager
from knowledge.chroma_knowledge_manager import knowledge_manager
from core.config import config
import os

print('🔧 Инициализация ChromaDB Knowledge Manager')
print('=' * 50)
print(f'📁 База знаний: {config.KNOWLEDGE_BASE_PATH}')
print(f'🔍 RAG включен: {config.ENABLE_RAG}')
print(f'📊 Chunk size: {config.RAG_CHUNK_SIZE}')
print(f'🎯 Top K: {config.RAG_TOP_K}')
print(f'🗄️ Векторная БД: ChromaDB')
print(f'💾 Путь хранения: {config.VECTOR_STORE_PATH}')

# Проверяем структуру директорий знаний
knowledge_path = '/content/ai-seo-architects/knowledge'
if os.path.exists(knowledge_path):
    for level in ['executive', 'management', 'operational']:
        level_path = os.path.join(knowledge_path, level)
        if os.path.exists(level_path):
            md_files = [f for f in os.listdir(level_path) if f.endswith('.md')]
            print(f'📚 {level:12}: {len(md_files):2d} файлов знаний')
        else:
            print(f'❌ {level:12}: директория не найдена')
else:
    print(f'❌ Директория знаний не найдена: {knowledge_path}')

# Инициализируем базы знаний для всех агентов в ChromaDB
print('\n🚀 Инициализация всех баз знаний в ChromaDB...')
print('=' * 50)
initialization_results = knowledge_manager.initialize_all_agents_knowledge()

successful_agents = [agent for agent, success in initialization_results.items() if success]
failed_agents = [agent for agent, success in initialization_results.items() if not success]

print(f'\n✅ Успешно в ChromaDB: {len(successful_agents)}/14 агентов')
print(f'❌ Ошибки: {len(failed_agents)} агентов')

if successful_agents:
    print(f'\n✅ Успешные агенты:')
    for i, agent in enumerate(successful_agents, 1):
        print(f'   {i:2d}. {agent}')
        
if failed_agents:
    print(f'\n❌ Проблемные агенты: {", ".join(failed_agents)}')

# Проверяем статистику ChromaDB
chroma_stats = knowledge_manager.get_stats()
total_docs = sum(store.get('documents_count', 0) for store in chroma_stats['stores'].values())

print(f'\n📊 СТАТИСТИКА CHROMADB:')
print('=' * 30)
print(f'💾 Коллекций ChromaDB: {chroma_stats["vector_stores_count"]}')
print(f'📄 Документов всего: {total_docs:,}')
print(f'🔗 OpenAI Embeddings: {"✅ Активны" if chroma_stats["embeddings_available"] else "❌ Недоступны"}')

# Сохраняем результаты для следующих ячеек
globals()['KNOWLEDGE_MANAGER'] = knowledge_manager
globals()['SUCCESSFUL_AGENTS'] = successful_agents
globals()['INITIALIZATION_RESULTS'] = initialization_results
globals()['CHROMA_STATS'] = chroma_stats

print(f'\n🎯 ChromaDB готова: {len(successful_agents) >= 10}')

In [ ]:
# ЯЧЕЙКА 4: Демонстрация ChromaDB поиска
import time

knowledge_manager = globals()['KNOWLEDGE_MANAGER']
successful_agents = globals()['SUCCESSFUL_AGENTS']

print('🔍 ДЕМОНСТРАЦИЯ CHROMADB RAG ПОИСКА')
print('=' * 60)

# Тестовые запросы для разных агентов
test_queries = [
    ('lead_qualification', 'Как квалифицировать лида с бюджетом 500000 рублей?'),
    ('technical_seo_auditor', 'Проведи технический SEO аудит сайта'),
    ('content_strategy', 'Создай контентную стратегию для B2B'),
    ('proposal_generation', 'Создай коммерческое предложение'),
    ('competitive_analysis', 'Анализ конкурентов в SEO')
]

chromadb_results = {}

for agent_id, query in test_queries:
    if agent_id in successful_agents:
        print(f'\n🤖 АГЕНТ: {agent_id}')
        print(f'❓ ЗАПРОС: {query}')
        print('-' * 50)
        
        try:
            # ChromaDB поиск через Knowledge Manager
            start_time = time.time()
            relevant_docs = knowledge_manager.search_knowledge(agent_id, query, k=3)
            search_time = time.time() - start_time
            
            if relevant_docs:
                total_chars = sum(len(doc.page_content) for doc in relevant_docs)
                print(f'✅ ChromaDB нашел: {len(relevant_docs)} документов')
                print(f'📊 Символов контекста: {total_chars:,}')
                print(f'⏱️ Время ChromaDB поиска: {search_time:.3f}с')
                
                # Показываем первый результат
                first_doc = relevant_docs[0]
                source = first_doc.metadata.get('source', 'unknown')
                content_preview = first_doc.page_content[:150]
                
                print(f'📄 Источник: {source}')
                print(f'📝 Превью: {content_preview}...')
                
                chromadb_results[agent_id] = {
                    'query': query,
                    'documents_found': len(relevant_docs),
                    'total_chars': total_chars,
                    'search_time': search_time,
                    'success': True
                }
                
            else:
                print('❌ ChromaDB: релевантные документы не найдены')
                chromadb_results[agent_id] = {
                    'query': query,
                    'documents_found': 0,
                    'success': False
                }
                
        except Exception as e:
            print(f'❌ Ошибка ChromaDB поиска: {str(e)[:60]}...')
            chromadb_results[agent_id] = {
                'query': query,
                'error': str(e),
                'success': False
            }
    else:
        print(f'⚠️ Агент {agent_id} не инициализирован в ChromaDB')

# Итоговая статистика ChromaDB
print('\n' + '=' * 60)
print('📊 СТАТИСТИКА CHROMADB RAG ПОИСКА:')
print('=' * 60)

successful_searches = len([r for r in chromadb_results.values() if r.get('success', False)])
total_docs_found = sum(r.get('documents_found', 0) for r in chromadb_results.values())
total_chars = sum(r.get('total_chars', 0) for r in chromadb_results.values())
search_times = [r.get('search_time', 0) for r in chromadb_results.values() if 'search_time' in r]
avg_search_time = sum(search_times) / max(1, len(search_times))

print(f'✅ Успешных ChromaDB поисков: {successful_searches}/{len(test_queries)}')
print(f'📄 ChromaDB нашел документов: {total_docs_found}')
print(f'📝 Символов контекста: {total_chars:,}')
print(f'⚡ Средняя скорость ChromaDB: {avg_search_time:.3f}с')

# Сохраняем результаты
globals()['CHROMADB_RESULTS'] = chromadb_results

print(f'\n🎯 ChromaDB RAG система работает отлично: {successful_searches >= 3}')

In [ ]:
# ЯЧЕЙКА 5: Инициализация агентов с ChromaDB RAG
from agents.operational.lead_qualification import LeadQualificationAgent
from agents.operational.technical_seo_auditor import TechnicalSEOAuditorAgent
from agents.operational.content_strategy import ContentStrategyAgent

print('🤖 ИНИЦИАЛИЗАЦИЯ АГЕНТОВ С CHROMADB RAG')
print('=' * 60)

successful_agents = globals()['SUCCESSFUL_AGENTS']

# Конфигурация ключевых агентов для демо
key_agents_config = [
    ('lead_qualification', LeadQualificationAgent, 'operational'),
    ('technical_seo_auditor', TechnicalSEOAuditorAgent, 'operational'),
    ('content_strategy', ContentStrategyAgent, 'operational')
]

# Инициализируем агентов с ChromaDB RAG
chromadb_agents = {}
successful_inits = 0

for agent_id, AgentClass, level in key_agents_config:
    if agent_id in successful_agents:
        try:
            # Создаем агента с ChromaDB RAG поддержкой
            agent = AgentClass(
                agent_id=agent_id,
                name=agent_id.replace('_', ' ').title(),
                agent_level=level,
                rag_enabled=True,
                knowledge_base=f'{level}/{agent_id}.md'
            )
            
            chromadb_agents[agent_id] = agent
            successful_inits += 1
            
            print(f'✅ {agent_id:30} | ChromaDB RAG: ✅')
            
        except Exception as e:
            print(f'❌ {agent_id:30} | Ошибка: {str(e)[:40]}')
    else:
        print(f'⚠️ {agent_id:30} | База знаний не готова')

print(f'\n📊 Результат: {successful_inits}/{len(key_agents_config)} агентов с ChromaDB RAG')

# Тест ChromaDB RAG с реальным агентом
if 'lead_qualification' in chromadb_agents:
    print(f'\n🧪 ТЕСТ CHROMADB RAG СИСТЕМЫ:')
    print('-' * 40)
    
    test_agent = chromadb_agents['lead_qualification']
    test_query = 'Квалификация enterprise клиента с бюджетом 1М рублей'
    
    try:
        # Получаем контекст через ChromaDB
        start_time = time.time()
        context = await test_agent.get_knowledge_context(test_query, k=3)
        context_time = time.time() - start_time
        
        print(f'📝 Тестовый запрос: {test_query}')
        print(f'⏱️ Время получения контекста: {context_time:.3f}с')
        print(f'📚 Длина контекста: {len(context):,} символов')
        
        if context:
            context_preview = context[:200].replace('\n', ' ')
            print(f'📄 Превью контекста: {context_preview}...')
            print(f'✅ ChromaDB RAG работает корректно!')
        else:
            print(f'❌ Контекст не получен из ChromaDB')
            
    except Exception as e:
        print(f'❌ Ошибка тестирования ChromaDB RAG: {e}')

# Сохраняем для следующих ячеек
globals()['CHROMADB_AGENTS'] = chromadb_agents
globals()['SUCCESSFUL_CHROMADB_AGENTS'] = successful_inits

print(f'\n🎯 ChromaDB агенты готовы: {successful_inits >= 2}')